In [1]:
import os
import pandas as pd
import numpy as np
from math import sqrt
from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [4]:
# 각자 작업 환경에 맞는 경로를 지정해주세요. Google Colab과 Jupyter환경에서 경로가 다를 수 있습니다.
path = '../data/movielens/'
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')

print(ratings_df.shape)
print(ratings_df.head())

(100836, 4)
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [5]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=1234)

print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


# sparse Matrix 만들기

sparse matrix = (user, movie)

### 1. 연산량이 많아서 시간이 오래걸리는 방법


In [7]:
user_ids = sorted(list(set(train_df['userId'].values)))
movie_ids = sorted(list(set(train_df['movieId'].values)))

print(f'유저 수: {len(user_ids)}, 영화 수: {len(movie_ids)}')
print(user_ids)
print(movie_ids)

유저 수: 610, 영화 수: 8938
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 

In [8]:
# Empty sparse matrix를 dataframe 형태로 준비
sparse_matrix = pd.DataFrame(index=movie_ids, columns=user_ids)
sparse_matrix

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# train_df의 movieId를 기준으로 groupBy
grouped = train_df.groupby('movieId')
grouped['userId'].count()

movieId
1         169
2          82
3          44
4           4
5          38
         ... 
193573      1
193579      1
193581      1
193587      1
193609      1
Name: userId, Length: 8938, dtype: int64

In [18]:
group

,userId,movieId,rating,timestamp
80358,508,1270,4.5,1268298253
32643,222,1270,4.5,1391350098
75556,477,1270,4.5,1200939690
18883,122,1270,5.0,1461561374
18302,116,1270,4.5,1337199782
...,...,...,...,...
24714,171,1270,5.0,866905762
83217,527,1270,4.0,1033173871
2501,19,1270,4.0,965703583
73463,474,1270,4.5,1081177274


In [15]:
group.transpose()

,80358,32643,75556,18883,18302,3266,62766,49089,12983,82599,...,89422,45695,45849,81635,38559,24714,83217,2501,73463,90079
userId,5.080000e+02,2.220000e+02,4.770000e+02,1.220000e+02,1.160000e+02,2.100000e+01,414.0,3.180000e+02,8.300000e+01,524.0,...,5.800000e+02,3.030000e+02,304.0,5.170000e+02,265.0,171.0,5.270000e+02,19.0,4.740000e+02,5.860000e+02
movieId,1.270000e+03,1.270000e+03,1.270000e+03,1.270000e+03,1.270000e+03,1.270000e+03,1270.0,1.270000e+03,1.270000e+03,1270.0,...,1.270000e+03,1.270000e+03,1270.0,1.270000e+03,1270.0,1270.0,1.270000e+03,1270.0,1.270000e+03,1.270000e+03
rating,4.500000e+00,4.500000e+00,4.500000e+00,5.000000e+00,4.500000e+00,5.000000e+00,5.0,2.500000e+00,3.500000e+00,5.0,...,3.500000e+00,3.500000e+00,5.0,5.000000e+00,5.0,5.0,4.000000e+00,4.0,4.500000e+00,4.500000e+00
timestamp,1.268298e+09,1.391350e+09,1.200940e+09,1.461561e+09,1.337200e+09,1.407618e+09,961514698.0,1.270753e+09,1.333842e+09,851609356.0,...,1.167793e+09,1.053303e+09,898641489.0,1.487957e+09,965314811.0,866905762.0,1.033174e+09,965703583.0,1.081177e+09,1.529900e+09


In [10]:
idx = 0
for movieId, group in grouped:
    if idx < 1:
        print(group)
        group_copied = group.transpose()
        group_copied.loc['userId'] = pd.to_numeric(group_copied.loc['userId'])
        print(group_copied)
        group_copied.columns = group_copied.loc['userId']
        print(group_copied)
        group_copied = group_copied.drop(['userId', 'movieId']).rename(index={'rating': movieId})
        print(group_copied)
        print(group_copied.columns)
        print(group_copied.index)
    else:
        continue
        idx += 1

       userId  movieId  rating   timestamp
6876       46        1     5.0   834787906
23663     161        1     4.0  1176751765
70844     453        1     5.0  1005966797
97043     604        1     3.0   832079851
24321     169        1     4.5  1059427918
...       ...      ...     ...         ...
81082     514        1     4.0  1533872400
68337     443        1     4.0  1501722482
71257     456        1     5.0   856883308
76227     480        1     3.0  1179178004
19179     124        1     4.0  1336584336

[169 rows x 4 columns]
                 6876          23663         70844        97043         24321  \
userId            46.0  1.610000e+02  4.530000e+02        604.0  1.690000e+02   
movieId            1.0  1.000000e+00  1.000000e+00          1.0  1.000000e+00   
rating             5.0  4.000000e+00  5.000000e+00          3.0  4.500000e+00   
timestamp  834787906.0  1.176752e+09  1.005967e+09  832079851.0  1.059428e+09   

                  58096         84780         37470   

                  32054        52380        70845        14818         14125  \
userId     2.200000e+02        340.0        453.0         94.0  9.100000e+01   
movieId    1.000000e+01         10.0         10.0         10.0  1.000000e+01   
rating     4.000000e+00          4.0          2.0          3.0  3.500000e+00   
timestamp  1.230056e+09  848666415.0  972621985.0  843406732.0  1.112713e+09   

                 23229         41462        8037          51195        89905  \
userId           160.0  2.820000e+02         57.0  3.310000e+02        584.0   
movieId           10.0  1.000000e+01         10.0  1.000000e+01         10.0   
rating             1.0  3.500000e+00          3.0  3.000000e+00          5.0   
timestamp  971196422.0  1.378497e+09  965798286.0  1.537158e+09  834987172.0   

           ...         27309         23866         39233        5877   \
userId     ...  1.860000e+02  1.660000e+02  2.740000e+02         42.0   
movieId    ...  1.000000e+01  1.000000e+01  1.000000

userId            64.0         555.0         438.0         480.0        470.0  \
19         3.500000e+00          3.0  3.000000e+00  1.500000e+00          3.0   
timestamp  1.161522e+09  980123949.0  1.105667e+09  1.179178e+09  849075545.0   

userId           136.0        284.0        42.0          249.0        321.0  \
19                 3.0          4.0          2.0  3.500000e+00          3.0   
timestamp  832449345.0  832786975.0  996256258.0  1.354107e+09  843212522.0   

userId     ...         381.0         509.0         156.0        117.0  \
19         ...  2.500000e+00  4.000000e+00  3.000000e+00          2.0   
timestamp  ...  1.165976e+09  1.436027e+09  1.106855e+09  844162892.0   

userId            200.0         377.0        56.0          222.0  \
19         3.500000e+00  2.000000e+00          5.0  3.500000e+00   
timestamp  1.229877e+09  1.340343e+09  835799219.0  1.391352e+09   

userId            357.0         534.0  
19         2.000000e+00  4.000000e+00  
timestamp  1.

                  50953         32260         27535        38902  \
userId     3.300000e+02  2.210000e+02  1.870000e+02        268.0   
movieId    2.900000e+01  2.900000e+01  2.900000e+01         29.0   
rating     3.000000e+00  4.500000e+00  4.000000e+00          5.0   
timestamp  1.285904e+09  1.119985e+09  1.161850e+09  940182766.0   

                  92642        85296         89035         19921  \
userId     5.990000e+02        555.0  5.760000e+02  1.320000e+02   
movieId    2.900000e+01         29.0  2.900000e+01  2.900000e+01   
rating     3.500000e+00          4.0  3.500000e+00  2.000000e+00   
timestamp  1.498501e+09  978841345.0  1.358151e+09  1.157924e+09   

                  48988         59236  ...         37974         14876  \
userId     3.180000e+02  3.870000e+02  ...  2.600000e+02  9.500000e+01   
movieId    2.900000e+01  2.900000e+01  ...  2.900000e+01  2.900000e+01   
rating     3.500000e+00  3.500000e+00  ...  4.000000e+00  5.000000e+00   
timestamp  1.413628e+0

userId           121.0        263.0        592.0        566.0         167.0  \
39                 4.0          4.0          4.0          4.0  2.000000e+00   
timestamp  847656062.0  941590574.0  837350111.0  849005720.0  1.154723e+09   

userId            480.0        266.0        597.0        174.0         95.0   \
39         2.500000e+00          2.0          4.0          4.0  4.000000e+00   
timestamp  1.179178e+09  945669397.0  941558116.0  848487101.0  1.043339e+09   

userId     ...        284.0        372.0         357.0        555.0  \
39         ...          3.0          3.0  4.500000e+00          4.0   
timestamp  ...  832786975.0  874417086.0  1.348612e+09  978746326.0   

userId            177.0        58.0          489.0        216.0         230.0  \
39         4.000000e+00          5.0  3.500000e+00          3.0  3.000000e+00   
timestamp  1.435891e+09  847718745.0  1.332773e+09  975212451.0  1.196305e+09   

userId           347.0  
39                 4.0  
timestamp  84

...

timestamp  1.171825e+09  1.354127e+09  
userId            182.0         23.0         469.0         105.0  \
1243       5.000000e+00  3.500000e+00          5.0  3.000000e+00   
timestamp  1.054780e+09  1.107342e+09  965425565.0  1.448196e+09   

userId            387.0        414.0         66.0          509.0  \
1243       3.000000e+00          5.0  5.000000e+00  4.000000e+00   
timestamp  1.182720e+09  961438248.0  1.099188e+09  1.436102e+09   

userId            599.0        57.0          51.0          68.0   \
1243       3.000000e+00          5.0  5.000000e+00  1.000000e+00   
timestamp  1.498525e+09  965796886.0  1.230931e+09  1.332629e+09   

userId            606.0         246.0  
1243       4.000000e+00  5.000000e+00  
timestamp  1.171825e+09  1.354127e+09  
Float64Index([182.0,  23.0, 469.0, 105.0, 387.0, 414.0,  66.0, 509.0, 599.0,
               57.0,  51.0,  68.0, 606.0, 246.0],
             dtype='float64', name='userId')
Index([1243, 'timestamp'], dtype='object')
       use

userId            264.0         387.0        422.0         29.0   \
userId     2.640000e+02  3.870000e+02        422.0  2.900000e+01   
movieId    1.250000e+03  1.250000e+03       1250.0  1.250000e+03   
rating     5.000000e+00  4.000000e+00          4.0  5.000000e+00   
timestamp  1.136978e+09  1.095041e+09  995235719.0  1.308025e+09   

userId            348.0        27.0          502.0        59.0         140.0  \
userId     3.480000e+02         27.0  5.020000e+02         59.0        140.0   
movieId    1.250000e+03       1250.0  1.250000e+03       1250.0       1250.0   
rating     4.500000e+00          4.0  5.000000e+00          5.0          5.0   
timestamp  1.378851e+09  962686417.0  1.111757e+09  953609923.0  942841937.0   

userId            448.0  ...         76.0          221.0         334.0  \
userId     4.480000e+02  ...  7.600000e+01  2.210000e+02  3.340000e+02   
movieId    1.250000e+03  ...  1.250000e+03  1.250000e+03  1.250000e+03   
rating     5.000000e+00  ...  3.0000

                  58178         38012         32358         88309  \
userId     3.810000e+02  2.600000e+02  2.210000e+02  5.700000e+02   
movieId    1.258000e+03  1.258000e+03  1.258000e+03  1.258000e+03   
rating     4.000000e+00  3.500000e+00  5.000000e+00  3.000000e+00   
timestamp  1.166891e+09  1.109408e+09  1.118247e+09  1.181478e+09   

                  47998         48831         5688          39441  \
userId     3.110000e+02  3.170000e+02  4.100000e+01  2.740000e+02   
movieId    1.258000e+03  1.258000e+03  1.258000e+03  1.258000e+03   
rating     4.000000e+00  5.000000e+00  5.000000e+00  4.500000e+00   
timestamp  1.057854e+09  1.430517e+09  1.458939e+09  1.171409e+09   

                  1713         45478  ...         16282         61177  \
userId     1.700000e+01        299.0  ...  1.050000e+02  4.000000e+02   
movieId    1.258000e+03       1258.0  ...  1.258000e+03  1.258000e+03   
rating     4.000000e+00          4.0  ...  4.000000e+00  4.500000e+00   
timestamp  1.322

                  59447        21230        387           25837         50014  \
userId     3.870000e+02        140.0          4.0  1.780000e+02  3.220000e+02   
movieId    1.265000e+03       1265.0       1265.0  1.265000e+03  1.265000e+03   
rating     3.500000e+00          3.0          4.0  4.500000e+00  4.000000e+00   
timestamp  1.094877e+09  949666815.0  964622684.0  1.164355e+09  1.217676e+09   

                  35719         76392         27603        84     \
userId     2.410000e+02  4.800000e+02  1.870000e+02          1.0   
movieId    1.265000e+03  1.265000e+03  1.265000e+03       1265.0   
rating     3.000000e+00  3.000000e+00  4.500000e+00          4.0   
timestamp  1.447536e+09  1.179162e+09  1.161864e+09  964983599.0   

                  52595  ...         67152         8816          21721  \
userId     3.440000e+02  ...  4.340000e+02  6.200000e+01  1.410000e+02   
movieId    1.265000e+03  ...  1.265000e+03  1.265000e+03  1.265000e+03   
rating     5.000000e+00  ...  5

KeyboardInterrupt: 

In [ ]:
for movieId, group in tqdm(grouped):
    group_copied = group
    group_copied = group_copied.transpose()
    group_copied.loc['userId'] = pd.to_numeric(group_copied.loc['userId'])
    group_copied.columns = group_copied.loc['userId']
    group_copied = group_copied.drop(['userId', 'movieId']).rename(index={'rating': movieId})

    sparse_matrix.update(group_copied)

### 2. 간단하고 효율적인 방법
- pandas dataframe의 unstack(계층적 인덱싱)


In [19]:
sparse_matrix = train_df.groupby('movieId').apply(lambda x: pd.Series(x['rating'].values, index=x['userId'])).unstack()
sparse_matrix.index.name = 'movieId'

sparse_matrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,NaN,2.5,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,NaN,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- `MEMO`: 0이 아닌 다른 숫자를 주는 것도 또 하나 시도해 볼 수 있는 실험이다. 

## 코사인 유사도 활용하기

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

def cossim_matrix(a, b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = a.index.values, index=a.index)

    return cossim_df

## Neighborhood-based 협업필터링 추천점수 계산하기

### Item-based

In [21]:
item_sparse_matrix = sparse_matrix.fillna(0)
item_sparse_matrix.shape

(8938, 610)

In [22]:
item_sparse_matrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,0.0,2.5,0.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# item과 item의 코사인 유사도
item_cossim_df = cossim_matrix(item_sparse_matrix, item_sparse_matrix)
item_cossim_df

,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.304336,0.267816,0.040259,0.221228,0.266544,0.149392,0.132943,0.182044,0.296838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.304336,1.000000,0.226138,0.052482,0.154783,0.209716,0.189420,0.068012,0.027945,0.303157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.267816,0.226138,1.000000,0.000000,0.306435,0.245555,0.368724,0.168267,0.253679,0.178219,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.040259,0.052482,0.000000,1.000000,0.095673,0.068708,0.205962,0.000000,0.000000,0.044835,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.221228,0.154783,0.306435,0.095673,1.000000,0.238683,0.343507,0.204088,0.222925,0.150729,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
193579,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
193581,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [24]:
# movieId: 8938개, userId: 610개
# train_df에 포함된 userId를 계산에 반영한다
userId_grouped = train_df.groupby('userId')
# index: userId, columns: total movieId
item_prediction_result_df = pd.DataFrame(index=list(userId_grouped.indices.keys()), columns=item_sparse_matrix.index)
item_prediction_result_df

movieId,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
for userId, group in tqdm(userId_grouped):
    # user가 rating한 movieId * 전체 movieId
    user_sim = item_cossim_df.loc[group['movieId']]
    # user가 rating한 movieId * 1
    user_rating = group['rating']
    # 전체 movieId * 1
    sim_sum = user_sim.sum(axis=0)

    # userId의 전체 rating predictions (8938 * 1)
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    item_prediction_result_df.loc[userId] = pred_ratings

/Users/ohyeji/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/610 [00:00<?, ?it/s]

In [26]:
item_prediction_result_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
1,4.20279,4.18783,4.17609,3.03272,4.04561,4.24019,3.9933,3.90547,3.70279,4.18855,...,0.979003,0,0,0,0,0,0,0,0,1.38273
2,3.19261,3.15207,2.46138,0.280903,2.61916,2.9754,1.85877,1.68768,1.09268,3.02235,...,0.387743,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,1.70885,2.32666
3,1.32315,1.20446,1.39635,0.217016,0.819716,1.59889,0.802514,0.755008,0.86708,1.55365,...,0,0,0,0,0,0,0,0,0,0
4,3.48994,3.48805,3.38864,3.15017,3.30488,3.50267,3.31794,2.90629,3.03413,3.41697,...,1.0335,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,0.505535,1.963
5,3.27916,3.1278,2.99947,2.56582,2.94179,3.16869,3.00793,2.67809,2.27668,3.01557,...,0.391638,0,0,0,0,0,0,0,0,0.562705
6,3.58597,3.57233,3.53615,3.33866,3.51444,3.5477,3.53826,3.42057,3.3406,3.55275,...,0.446497,0,0,0,0,0,0,0,0,0.672442
7,3.36009,3.25338,3.24516,2.22581,3.2114,3.33808,3.1832,2.8918,2.89194,3.3515,...,0.872628,0.3554,0.3554,0.3554,0.3554,0.3554,0.3554,0.3554,0.3554,2.50752
8,3.26123,3.22385,3.03702,2.58129,2.94548,3.21505,3.06126,2.6713,2.37,3.17426,...,0.597653,0,0,0,0,0,0,0,0,0.730709
9,2.80797,2.62156,2.28209,0.767416,2.25043,2.64179,2.04582,1.59945,1.07072,2.63652,...,0.458811,0,0,0,0,0,0,0,0,0
10,3.07042,3.04355,2.85983,0.908598,2.97965,2.905,2.75228,2.24064,1.58879,3.05624,...,0.63253,1.21136,1.21136,1.21136,1.21136,1.21136,1.21136,1.21136,1.21136,2.33335


In [34]:
user_sim

,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
6294,0.126311,0.154142,0.038764,0.0,0.049955,0.201797,0.000000,0.000000,0.000000,0.246324,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
148626,0.123049,0.165150,0.054465,0.0,0.074149,0.182987,0.000000,0.000000,0.021681,0.086002,...,0.000000,0.264443,0.264443,0.264443,0.264443,0.264443,0.264443,0.264443,0.264443,0.0
111360,0.109545,0.113282,0.034396,0.0,0.041719,0.167778,0.000000,0.000000,0.000000,0.104107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5323,0.079107,0.132934,0.000000,0.0,0.000000,0.189852,0.000000,0.000000,0.000000,0.117006,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
97752,0.121847,0.147378,0.036918,0.0,0.067166,0.169270,0.000000,0.000000,0.026671,0.055083,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144,0.158466,0.044723,0.090335,0.0,0.050901,0.108734,0.064473,0.000000,0.000000,0.021832,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
156371,0.075384,0.078616,0.000000,0.0,0.000000,0.107210,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
31086,0.096538,0.000000,0.000000,0.0,0.000000,0.137296,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [36]:
user_sim

,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
6294,0.126311,0.154142,0.038764,0.0,0.049955,0.201797,0.000000,0.000000,0.000000,0.246324,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
148626,0.123049,0.165150,0.054465,0.0,0.074149,0.182987,0.000000,0.000000,0.021681,0.086002,...,0.000000,0.264443,0.264443,0.264443,0.264443,0.264443,0.264443,0.264443,0.264443,0.0
111360,0.109545,0.113282,0.034396,0.0,0.041719,0.167778,0.000000,0.000000,0.000000,0.104107,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5323,0.079107,0.132934,0.000000,0.0,0.000000,0.189852,0.000000,0.000000,0.000000,0.117006,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
97752,0.121847,0.147378,0.036918,0.0,0.067166,0.169270,0.000000,0.000000,0.026671,0.055083,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144,0.158466,0.044723,0.090335,0.0,0.050901,0.108734,0.064473,0.000000,0.000000,0.021832,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
156371,0.075384,0.078616,0.000000,0.0,0.000000,0.107210,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
31086,0.096538,0.000000,0.000000,0.0,0.000000,0.137296,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


### User-based

In [38]:
user_sparse_matrix = sparse_matrix.fillna(0).transpose()

In [39]:
user_sparse_matrix.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,190219,191005,193565,193567,193571,193573,193579,193581,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
user_sparse_matrix.shape

(610, 8938)

In [41]:
user_cossim_df = cossim_matrix(user_sparse_matrix, user_sparse_matrix)
user_cossim_df

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.016665,0.070570,0.160438,0.075410,0.087404,0.123664,0.069225,0.037416,0.009900,...,0.056163,0.124896,0.162324,0.050852,0.101583,0.128198,0.240652,0.225897,0.063984,0.095228
2,0.016665,1.000000,0.000000,0.004295,0.020560,0.030996,0.027726,0.000000,0.000000,0.057112,...,0.151666,0.019379,0.006645,0.000000,0.000000,0.028180,0.000000,0.046286,0.033522,0.090288
3,0.070570,0.000000,1.000000,0.002677,0.000000,0.003477,0.000000,0.000000,0.000000,0.000000,...,0.002878,0.002174,0.027609,0.000000,0.000000,0.012090,0.000000,0.023927,0.000000,0.018332
4,0.160438,0.004295,0.002677,1.000000,0.121648,0.093634,0.106495,0.047930,0.000000,0.034281,...,0.065328,0.116656,0.251105,0.056396,0.075940,0.187236,0.102819,0.109515,0.038805,0.086941
5,0.075410,0.020560,0.000000,0.121648,1.000000,0.164390,0.075932,0.302418,0.000000,0.000000,...,0.090020,0.306634,0.103646,0.145354,0.119800,0.071333,0.064705,0.111755,0.159946,0.049236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.128198,0.028180,0.012090,0.187236,0.071333,0.076207,0.159508,0.051044,0.034675,0.066999,...,0.123464,0.083967,0.238107,0.068227,0.110706,1.000000,0.101661,0.218106,0.073027,0.164355
607,0.240652,0.000000,0.000000,0.102819,0.064705,0.121999,0.167129,0.167671,0.000000,0.010033,...,0.064124,0.196274,0.176603,0.111658,0.116814,0.101661,1.000000,0.233010,0.142659,0.114225
608,0.225897,0.046286,0.023927,0.109515,0.111755,0.144931,0.274492,0.166382,0.053714,0.054227,...,0.134966,0.183543,0.198187,0.130758,0.159466,0.218106,0.233010,1.000000,0.107489,0.250085


In [42]:
movieId_grouped = train_df.groupby('movieId')
user_prediction_result_df = pd.DataFrame(index=list(movieId_grouped.indices.keys()), columns=user_sparse_matrix.index)
user_prediction_result_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
for movieId, group in tqdm(movieId_grouped):
    user_sim = user_cossim_df.loc[group['userId']]
    user_rating = group['rating']
    sim_sum = user_sim.sum(axis=0)

    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    user_prediction_result_df.loc[movieId] = pred_ratings

# return user_prediction_result_df.transpose()

/Users/ohyeji/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/8938 [00:00<?, ?it/s]

In [44]:
# 전체 user가 모든 movieId에 매긴 평점
print(item_prediction_result_df.head())
print(user_prediction_result_df.transpose().head())

user_prediction_result_df = user_prediction_result_df.transpose()

movieId   1        2        3         4         5        6         7       \
1        4.20279  4.18783  4.17609   3.03272   4.04561  4.24019    3.9933   
2        3.19261  3.15207  2.46138  0.280903   2.61916   2.9754   1.85877   
3        1.32315  1.20446  1.39635  0.217016  0.819716  1.59889  0.802514   
4        3.48994  3.48805  3.38864   3.15017   3.30488  3.50267   3.31794   
5        3.27916   3.1278  2.99947   2.56582   2.94179  3.16869   3.00793   

movieId    8        9        10      ...    190219    191005    193565  \
1         3.90547  3.70279  4.18855  ...  0.979003         0         0   
2         1.68768  1.09268  3.02235  ...  0.387743   1.70885   1.70885   
3        0.755008  0.86708  1.55365  ...         0         0         0   
4         2.90629  3.03413  3.41697  ...    1.0335  0.505535  0.505535   
5         2.67809  2.27668  3.01557  ...  0.391638         0         0   

movieId    193567    193571    193573    193579    193581    193587    193609  
1           

## RMSE로 추천시스템 성능 평가하기

In [45]:
test_df.head()

,userId,movieId,rating,timestamp
99731,610,3527,5.0,1479545223
97583,606,1250,3.5,1171376891
38197,262,213,5.0,840310907
11474,68,69406,3.0,1261622505
34105,232,4728,3.0,1218166950


In [46]:
def evaluate(test_df, prediction_result_df):
    groups_with_movie_ids = test_df.groupby(by='movieId')
    groups_with_user_ids = test_df.groupby(by='userId')
    intersection_movie_ids = sorted(list(set(list(prediction_result_df.columns)).intersection(set(list(groups_with_movie_ids.indices.keys())))))
    intersection_user_ids = sorted(list(set(list(prediction_result_df.index)).intersection(set(groups_with_user_ids.indices.keys()))))

    print(len(intersection_movie_ids))
    print(len(intersection_user_ids))

    compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_movie_ids]
    # compressed_prediction_df

    # test_df에 대해서 RMSE 계산
    grouped = test_df.groupby(by='userId')
    result_df = pd.DataFrame(columns=['rmse'])
    for userId, group in tqdm(grouped):
      if userId in intersection_user_ids:
          pred_ratings = compressed_prediction_df.loc[userId][compressed_prediction_df.loc[userId].index.intersection(list(group['movieId'].values))]
          pred_ratings = pred_ratings.to_frame(name='rating').reset_index().rename(columns={'index':'movieId','rating':'pred_rating'})
          actual_ratings = group[['rating', 'movieId']].rename(columns={'rating':'actual_rating'})

          final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['movieId'])
          final_df = final_df.round(4) # 반올림

          # if not final_df.empty:
          #     rmse = sqrt(mean_squared_error(final_df['rating_actual'], final_df['rating_pred']))
          #     result_df.loc[userId] = rmse
          #     # print(userId, rmse)

    return final_df

In [47]:
evaluate(test_df, user_prediction_result_df)

4385
610


/Users/ohyeji/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/610 [00:00<?, ?it/s]

,actual_rating,movieId,pred_rating
0,5.0,3527,3.28567
1,3.5,84772,2.21159
2,3.5,103141,2.83544
3,4.0,81132,0.450808
4,4.5,130634,1.02244
...,...,...,...
218,4.0,106100,2.86823
219,4.0,111759,3.51395
220,1.0,4852,0.202402
221,3.0,2628,2.93777


In [48]:
evaluate(test_df, item_prediction_result_df)

4385
610


/Users/ohyeji/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/610 [00:00<?, ?it/s]

,actual_rating,movieId,pred_rating
0,5.0,3527,3.79741
1,3.5,84772,3.73065
2,3.5,103141,3.6897
3,4.0,81132,3.33402
4,4.5,130634,3.4512
...,...,...,...
218,4.0,106100,3.72969
219,4.0,111759,3.75842
220,1.0,4852,3.64888
221,3.0,2628,3.81238


In [49]:
result_df = evaluate(test_df, user_prediction_result_df)
print(result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

4385
610


/Users/ohyeji/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/610 [00:00<?, ?it/s]

     actual_rating  movieId pred_rating
0              5.0     3527     3.28567
1              3.5    84772     2.21159
2              3.5   103141     2.83544
3              4.0    81132    0.450808
4              4.5   130634     1.02244
..             ...      ...         ...
218            4.0   106100     2.86823
219            4.0   111759     3.51395
220            1.0     4852    0.202402
221            3.0     2628     2.93777
222            5.0     1953     3.17222

[223 rows x 3 columns]
RMSE: 1.6949489761800296


In [50]:
result_df = evaluate(test_df, item_prediction_result_df)
print(result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

4385
610


/Users/ohyeji/anaconda3/envs/tf2/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/610 [00:00<?, ?it/s]

     actual_rating  movieId pred_rating
0              5.0     3527     3.79741
1              3.5    84772     3.73065
2              3.5   103141      3.6897
3              4.0    81132     3.33402
4              4.5   130634      3.4512
..             ...      ...         ...
218            4.0   106100     3.72969
219            4.0   111759     3.75842
220            1.0     4852     3.64888
221            3.0     2628     3.81238
222            5.0     1953     3.86646

[223 rows x 3 columns]
RMSE: 0.8145193961484049
